# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [4]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [5]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [17]:
quantities = data.groupby(['CustomerID','ProductName'])['Quantity'].sum()
quantities

CustomerID  ProductName                  
33          Apricots - Dried                  1
            Assorted Desserts                 1
            Bandage - Flexible Neon           1
            Bar Mix - Pina Colada, 355 Ml     1
            Beans - Kidney, Canned            1
                                             ..
98200       Vol Au Vents                     50
            Wasabi Powder                    25
            Wine - Fume Blanc Fetzer         25
            Wine - Hardys Bankside Shiraz    25
            Yogurt - French Vanilla          25
Name: Quantity, Length: 63628, dtype: int64

In [15]:
type(quantities)

pandas.core.series.Series

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [22]:
pivot_table = data.pivot_table(index='ProductName', columns='CustomerID', values='Quantity', fill_value=0)
pivot_table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [23]:
# Calculate the pairwise distances between customers using the 'euclidean' metric
distances = pdist(pivot_table.values.T, metric='euclidean')
distances

array([ 10.72380529,   9.64365076,   9.89949494, ..., 266.9269563 ,
       266.9269563 , 269.25824036])

In [24]:
# Create a square matrix from the pairwise distances
similarity_matrix = pd.DataFrame(squareform(distances), index=pivot_table.columns, columns=pivot_table.columns)
similarity_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,10.723805,9.643651,9.899495,10.440307,10.148892,10.723805,10.246951,10.344080,10.148892,...,206.963765,194.766527,199.145675,183.463893,202.259734,198.075743,203.921554,197.949489,218.824130,203.798921
200,10.723805,0.000000,10.677078,10.723805,11.489125,11.045361,10.770330,10.954451,10.677078,10.488088,...,206.269242,194.285872,198.423789,182.543146,202.044550,197.729613,202.723950,197.349943,218.625250,203.953426
264,9.643651,10.677078,0.000000,10.344080,10.198039,10.488088,10.677078,10.583005,10.488088,10.488088,...,206.366179,194.517351,199.153207,182.789496,201.772149,196.817174,203.315026,197.324606,218.029814,203.806281
356,9.899495,10.723805,10.344080,0.000000,10.630146,10.535654,10.630146,9.949874,10.630146,10.440307,...,206.726873,194.643263,199.025124,182.923481,202.264678,197.575302,204.293906,197.322072,218.142155,203.681123
412,10.440307,11.489125,10.198039,10.630146,0.000000,10.677078,10.677078,10.198039,10.770330,10.392305,...,206.985507,193.889144,199.168271,182.805908,202.034650,197.339808,204.066166,197.466453,217.928887,202.960587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,198.075743,197.729613,196.817174,197.575302,197.339808,197.324606,197.339808,197.441130,197.451260,197.699267,...,266.926956,252.487623,265.753645,246.221445,251.246891,0.000000,252.487623,252.487623,278.388218,276.134025
98069,203.921554,202.723950,203.315026,204.293906,204.066166,204.051464,203.821000,203.919102,203.683578,203.678668,...,273.861279,264.575131,268.095132,263.391344,256.173769,252.487623,0.000000,254.950976,276.134025,264.575131
98159,197.949489,197.349943,197.324606,197.322072,197.466453,197.451260,197.466453,197.694208,197.197870,197.572771,...,266.926956,250.000000,258.602011,256.173769,261.007663,252.487623,254.950976,0.000000,266.926956,266.926956


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [26]:
target_customer_id = 200 # here i am taking the 200 customerid

# Get the similarity scores for the target customer
target_customer_scores = similarity_matrix.loc[target_customer_id]
target_customer_scores


CustomerID
33        10.723805
200        0.000000
264       10.677078
356       10.723805
412       11.489125
            ...    
97928    197.729613
98069    202.723950
98159    197.349943
98185    218.625250
98200    203.953426
Name: 200, Length: 1000, dtype: float64

In [27]:
similar_customers = target_customer_scores.drop(target_customer_id).sort_values(ascending=False)
similar_customers

CustomerID
96560    219.253278
91443    218.924645
98185    218.625250
97324    218.281928
91777    216.831732
            ...    
3909      10.148892
3317      10.099505
3253      10.099505
1072       9.797959
1920       9.591663
Name: 200, Length: 999, dtype: float64

In [29]:
# Get the top 5 most similar customers
top_similar_customers = similar_customers.head(5)
top_similar_customers

CustomerID
96560    219.253278
91443    218.924645
98185    218.625250
97324    218.281928
91777    216.831732
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [30]:
# Get the CustomerIDs of the top 5 similar customers
similar_customer_ids = top_similar_customers.index.tolist()
similar_customer_ids

[96560, 91443, 98185, 97324, 91777]

In [31]:
# Filter the data for the top 5 similar customers
similar_customers_data = data[data['CustomerID'].isin(similar_customer_ids)]
similar_customers_data

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
57,97324,Darrick,Williamson,5653021,229,Bread - Hot Dog Buns,25
58,97324,Darrick,Williamson,6196374,229,Bread - Hot Dog Buns,25
94,98185,Katina,Moyer,4759015,229,Bread - Hot Dog Buns,25
806,91777,Ramona,Cabrera,2203,102,"Rosemary - Primerba, Paste",24
1125,91777,Ramona,Cabrera,6252822,17,"Wine - White, Mosel Gold",24
...,...,...,...,...,...,...,...
67896,91777,Ramona,Cabrera,2716494,351,Hot Chocolate - Individual,24
68114,98185,Katina,Moyer,3306355,292,Turkey - Oven Roast Breast,25
68211,97324,Darrick,Williamson,6041846,106,Pickerel - Fillets,25
68263,98185,Katina,Moyer,6752054,106,Pickerel - Fillets,25


In [32]:
# Aggregate the purchase records by ProductName and sum the Quantity
product_purchases = similar_customers_data.groupby('ProductName')['Quantity'].sum()
product_purchases

ProductName
Anchovy Paste - 56 G Tube            25
Appetizer - Mini Egg Roll, Shrimp    24
Appetizer - Mushroom Tart            25
Appetizer - Sausage Rolls            49
Apricots - Halves                    25
                                     ..
Wine - Wyndham Estate Bin 777        49
Wonton Wrappers                      25
Yeast Dry - Fermipan                 25
Yogurt - Blueberry, 175 Gr           48
Yogurt - French Vanilla              25
Name: Quantity, Length: 270, dtype: int64

In [33]:
# Rank the products in descending order by quantity
ranked_products = product_purchases.sort_values(ascending=False)
ranked_products

ProductName
Cream Of Tartar                 100
Pasta - Penne, Rigate, Dry       99
Durian Fruit                     99
Shrimp - 31/40                   99
Bandage - Flexible Neon          98
                               ... 
Lemonade - Natural, 591 Ml       24
Placemat - Scallop, White        24
Langers - Ruby Red Grapfruit     24
Lamb - Whole, Fresh              24
Ocean Spray - Ruby Red           24
Name: Quantity, Length: 270, dtype: int64

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [36]:
# Merge the ranked products DataFrame with the customer product matrix on the 'ProductName' field
merged_data = pd.merge(ranked_products, pivot_table, on='ProductName')
merged_data

,Quantity,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Cream Of Tartar,100,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Pasta - Penne, Rigate, Dry",99,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,25,0
Durian Fruit,99,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
Shrimp - 31/40,99,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,25,0,0
Bandage - Flexible Neon,98,1,0,1,0,0,0,0,0,0,...,0,0,25,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Lemonade - Natural, 591 Ml",24,0,1,1,0,0,0,0,0,0,...,0,0,0,0,25,0,0,0,0,0
"Placemat - Scallop, White",24,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Langers - Ruby Red Grapfruit,24,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,25,0,0,0,0


In [37]:
# Filter for records where the chosen customer has not purchased the product
recommended_products = merged_data[merged_data[target_customer_id] == 0]
recommended_products

,Quantity,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Cream Of Tartar,100,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Pasta - Penne, Rigate, Dry",99,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,25,0
Durian Fruit,99,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
Bandage - Flexible Neon,98,1,0,1,0,0,0,0,0,0,...,0,0,25,0,0,0,0,0,0,0
Chicken - Soup Base,98,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Squid - Tubes / Tenticles 10/20,24,1,0,0,0,0,0,0,1,0,...,25,0,0,0,0,0,0,25,0,0
Corn Meal,24,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,25,0,0,0,0
"Placemat - Scallop, White",24,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
 recommended_products.head(5)


,Quantity,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Cream Of Tartar,100,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Pasta - Penne, Rigate, Dry",99,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,25,0
Durian Fruit,99,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
Bandage - Flexible Neon,98,1,0,1,0,0,0,0,0,0,...,0,0,25,0,0,0,0,0,0,0
Chicken - Soup Base,98,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [39]:
# Create an empty dictionary to hold recommendations for all customers
recommendations = {}

In [40]:
# Create a list of unique CustomerIDs
customer_list = pivot_table.columns.tolist()
customer_list

[33,
 200,
 264,
 356,
 412,
 464,
 477,
 639,
 649,
 669,
 694,
 756,
 883,
 891,
 1008,
 1034,
 1066,
 1072,
 1336,
 1428,
 1435,
 1534,
 1577,
 1594,
 1754,
 1839,
 1920,
 2187,
 2329,
 2503,
 2556,
 2566,
 2582,
 2617,
 2686,
 2754,
 2776,
 2902,
 2915,
 2939,
 3074,
 3253,
 3267,
 3305,
 3317,
 3472,
 3531,
 3535,
 3544,
 3885,
 3903,
 3909,
 4261,
 4479,
 4595,
 4644,
 4723,
 5104,
 5177,
 5183,
 5224,
 5230,
 5434,
 5678,
 5739,
 5832,
 5921,
 5968,
 5986,
 5997,
 6001,
 6185,
 6317,
 6352,
 6367,
 6447,
 6543,
 6734,
 6817,
 6851,
 7021,
 7134,
 7223,
 7543,
 7606,
 7623,
 7772,
 7827,
 7987,
 7999,
 8316,
 8367,
 8372,
 8539,
 8711,
 8917,
 8962,
 9047,
 9132,
 9260,
 9302,
 9533,
 9546,
 9597,
 9771,
 9804,
 10016,
 10094,
 10114,
 10224,
 10311,
 10418,
 10739,
 11023,
 11057,
 11083,
 11102,
 11253,
 11325,
 11380,
 11441,
 11443,
 11614,
 11628,
 11744,
 11883,
 11933,
 12059,
 12206,
 12232,
 12381,
 12529,
 12561,
 12596,
 12598,
 12852,
 12888,
 13062,
 13118,
 13338,
 

In [41]:
for customer_id in customer_list:
    target_customer_id = customer_id
    target_customer_scores = similarity_matrix.loc[target_customer_id]
    similar_customers = target_customer_scores.drop(target_customer_id).sort_values(ascending=False)
    top_similar_customers = similar_customers.head(5)

In [42]:
# Store the top 5 recommended products in the recommendations dictionary
recommendations[customer_id] = recommended_products.head(5)


In [43]:
# Display the recommendations for all customers
for customer_id, recommended_products in recommendations.items():
    print("Customer ID:", customer_id)
    print(recommended_products)
    print()

Customer ID: 98200
                            Quantity  33  200  264  356  412  464  477  639  \
ProductName                                                                   
Cream Of Tartar                  100   0    0    0    0    0    0    1    0   
Pasta - Penne, Rigate, Dry        99   0    0    0    0    0    0    0    0   
Durian Fruit                      99   0    0    0    0    0    0    0    0   
Bandage - Flexible Neon           98   1    0    1    0    0    0    0    0   
Chicken - Soup Base               98   0    0    0    0    1    0    0    0   

                            649  ...  97697  97753  97769  97793  97900  \
ProductName                      ...                                      
Cream Of Tartar               0  ...      0     25      0      0      0   
Pasta - Penne, Rigate, Dry    0  ...      0      0      0     25      0   
Durian Fruit                  0  ...      0      0      0      0      0   
Bandage - Flexible Neon       0  ...      0      0  

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.